In [ ]:
import random
import numpy as np


class Function:
    def __init__(self, value, delt=0.001):
        self.value = value

        def grad(x):
            array = []
            for i in range(len(x)):
                x[i] += delt
                first_val = value(x)
                x[i] -= 2 * delt
                second_val = value(x)
                x[i] += delt
                array.append((first_val - second_val) / (2 * delt))
            return np.array(array)

        self.grad = grad


wolfe_cond_template = lambda c1, c2, x, func, gk: lambda a, b: not (
        (func.value(x - ((a + b) / 2) * gk) <= (func.value(x) + c1 * ((a + b) / 2) * np.dot(gk, -gk))) and (
        np.dot(func.grad(x - ((a + b) / 2) * gk), -gk) >= c2 * np.dot(gk, -gk)))

wolfe_cond = lambda: ""


def grad_desc(func, lr, x, eps, is_wolfe=False, left_wolfe_bound=0.1, right_wolfe_bound=0.9, need_points=True,
              max_num_of_step=1000):
    global wolfe_cond
    num_of_steps = 0
    points = [x]
    while True:
        prev_x = x
        grad = func.grad(x)
        if is_wolfe:
            wolfe_cond = wolfe_cond_template(left_wolfe_bound, right_wolfe_bound, x, func, grad)
        x = x - lr(Function(lambda a: func.value(x - a * grad)), is_wolfe) * grad
        if need_points:
            points.append(x)
        num_of_steps += 1
        if abs(func.value(x) - func.value(prev_x)) < eps or num_of_steps >= max_num_of_step:
            break

    if need_points: return np.array(points)
    return num_of_steps


def right_border_calc(func):
    right_start = 0.25
    zero = func.value(0)
    while zero >= func.value(right_start):
        right_start *= 2

    return right_start


def dichotomy(func, a_1, a_2, eps, delt, is_wolfe=False):
    cond = lambda a, b: abs(a - b) >= eps
    if is_wolfe:
        cond = wolfe_cond
    while cond(a_1, a_2):
        new_a_1 = (a_1 + a_2) / 2 - delt
        new_a_2 = (a_1 + a_2) / 2 + delt

        if func.value(new_a_2) > func.value(new_a_1):
            a_2 = new_a_2
        else:
            a_1 = new_a_1

    return (a_1 + a_2) / 2


def func_generator(n, k):
    vals = [random.uniform(1.0, k) for _ in range(n)]
    vals.sort()
    vals[0] = 1
    vals[n - 1] = k
    q, r = np.linalg.qr(np.random.rand(n, n))
    matr = np.matmul(np.matmul(q, np.diag(vals)), np.transpose(q))
    return matr


class GeneratedFunctionCalculator(Function):
    def __init__(self, matr):
        super().__init__(lambda x: sum(matr[i][j] * x[i] * x[j] for i in range(len(x)) for j in range(len(x))))
        self.matr = matr

In [ ]:
import matplotlib.pyplot as plt


def lr_wrapper(eps, delt):
    return lambda func, is_wolfe=False: dichotomy(func, 0, right_border_calc(func), eps, delt, is_wolfe)


def func1(x):
    return 5 * (x[0] - 4) ** 2 + (5 - x[1]) ** 2


def func2(x):
    return 3 * x[0] ** 2 + x[1] ** 2 / 5


def func3(x):
    return (x[0] - 6) * (x[1] + 8) + 2 * x[0] ** 2 + 4 * x[1] ** 2


def visualise_points(func, points):
    XX = np.linspace(min(points[-1][0], points[0][0]) - 30, max(points[-1][0], points[0][0]) + 30, 500)
    YY = np.linspace(min(points[-1][1], points[0][1]) - 30, max(points[-1][1], points[0][1]) + 30, 500)
    X, Y = np.meshgrid(XX, YY)
    plt.contour(X, Y, func.value([X, Y]), 20)

    plt.plot(points[:, 0], points[:, 1])
    plt.plot(points[0][0], points[0][1], 'og')
    plt.plot(points[-1][0], points[-1][1], 'or')
    plt.show()


functions = [Function(func) for func in [func1, func2, func3]]

### 4(a) Сходимость с постоянным шагом

In [ ]:
def test_convergence_const_lr(func):
    lr = 0.01
    eps = 0.001
    x = np.array((20., 20.))

    print('Const rate: ' + str(lr) + ' | Start x: ' + str(x) + ' Epsilon: ' + str(eps))
    points = grad_desc(func, lambda *_: lr, x, eps)
    print('Count steps: ' + str(len(points)))
    print('Final point: ' + str(points[-1]))
    visualise_points(func, points)
    print()


for func in functions:
    test_convergence_const_lr(func)

### 4(b) Сравнение количества

In [ ]:
def compare_count_calculations(func):
    lr = 0.01
    eps = 0.01
    delt = 0.001
    x = np.array((20., 20.))

    print('Const rate: ' + str(lr) + ' | Start x: ' + str(x) + ' Epsilon: ' + str(eps))
    points_const_lr = grad_desc(func, lambda *_: lr, x, eps)
    print('Count steps with const rate: ' + str(len(points_const_lr)))
    print('Final point with const rate: ' + str(points_const_lr[-1]))
    visualise_points(func, points_const_lr)

    print('Start x: ' + str(x) + ' Epsilon: ' + str(eps) + ' | Delta: ' + str(delt))
    points_dichotomy = grad_desc(func, lr_wrapper(eps, delt), x, eps)
    print('Count steps with dichotomy: ' + str(len(points_dichotomy)))
    print('Final point with dichotomy: ' + str(points_dichotomy[-1]))
    visualise_points(func, points_dichotomy)
    print()


for func in functions:
    compare_count_calculations(func)

### 4(c) Тест на разные начальные точки

In [ ]:
def test_different_start_points(func):
    start_x = [np.array(x) for x in [(20., 20.), (-5., -10.), (-8., 10.)]]
    for x in start_x:
        lr = 0.01
        eps = 0.01
        delt = 0.001

        print('Const rate: ' + str(lr) + ' | Start x: ' + str(x) + ' Epsilon: ' + str(eps))
        points_const_lr = grad_desc(func, lambda *_: lr, x, eps)
        print('Count steps with const rate: ' + str(len(points_const_lr)))
        print('Final point with const rate: ' + str(points_const_lr[-1]))
        visualise_points(func, points_const_lr)

        print('Start x: ' + str(x) + ' Epsilon: ' + str(eps) + ' | Delta: ' + str(delt))
        points_dichotomy = grad_desc(func, lr_wrapper(eps, delt), x, eps)
        print('Count steps with dichotomy: ' + str(len(points_dichotomy)))
        print('Final point with dichotomy: ' + str(points_dichotomy[-1]))
        visualise_points(func, points_dichotomy)
        print()


for func in functions:
    test_different_start_points(func)

### 4(d) Исследование влияния нормализации

### 6 Исследование зависимости числа итераций от размерности пространства и числа обусловленности функции